In [ ]:
import 
filename = sys.argv[0]

In [25]:
import fasttext as ft
model = ft.load_model('../data/model_unprocessed24-03-2020.bin')

In [26]:
import pandas as pd
df = pd.read_csv('../data/corpus_check.csv')

In [27]:
df

,siren,legal_name,id,url_article,corpus
0,419838529,IPSEN,8,http://www.boursier.com/actions/actualites/new...,Ipsen lorgne les peptides de PeptiMimesis\n
1,419838529,IPSEN,2894,http://www.boursier.com/actions/actualites/new...,Ipsen : accord important avec Probi\n \n\npubl...
2,419838529,IPSEN,3057,http://www.boursier.com/actions/actualites/new...,La médecine générale d'Ipsen en panne au 1er t...
3,419838529,IPSEN,4208,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
4,419838529,IPSEN,5284,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
...,...,...,...,...,...
495,395030844,SANOFI,41863,https://www.lemondedudroit.fr/deals/57157-clif...,Clifford Chance conseil de banques sur l'émiss...
496,395030844,SANOFI,42528,https://www.lesechos.fr/industrie-services/pha...,Sanofi fragilisé par le recul de son activité ...
497,395030844,SANOFI,43721,https://www.lesechos.fr/industrie-services/pha...,Sanofi prépare son entrée sur le marché de l'a...
498,395030844,SANOFI,43756,https://www.mypharma-editions.com/sanofi-le-do...,» Sanofi : le Docteur Jacques Volckmann nommé ...


In [28]:
import re
import contractions
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import FrenchStemmer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger

nlp = spacy.load('fr')
nlp.max_length = 68111459
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')
translator = str.maketrans('', '', string.punctuation)


2020-03-31 09:52:23,614 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-03-31 09:52:23,615 - spacy_lefff.lefff - INFO - Token lefff_lemma already registered
2020-03-31 09:52:23,617 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-03-31 09:52:24,459 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [29]:
translator = str.maketrans('', '', string.punctuation)

# Tokenize text
def preprocessing(doc):
        result = []
        doc = doc.replace('\n', ' ')
        # To lowercase 
        doc = doc.lower()
        # Remove numbers
        doc = re.sub("\d+", "", doc)
        doc = doc.replace('m€', '')
        doc = doc.replace('k€', '')    
        sentences = sent_tokenize(doc)
        # Lemmentization (same as stemming but based on knowledge base)
        for sent in sentences: 
            # Delete punctuation
            sent = sent.translate(translator)
            lemmentization = []
            doc_ = nlp(sent,disable = ['ner', 'parser'])
            for d in doc_:
                lemmentization.append(d.lemma_)
            lemmentization.append('\n')
            res_ = " ".join(lemmentization)
            result.append(res_)
        return result 

def test(dataframe,index=0):
    f = open('' + str(index) + '.txt' , 'w')
    for i,row in dataframe.iterrows():
        result = "".join(preprocessing((row['text'])))
        f.write(result)  # python will convert \n to os.linesep
    f.close()  

In [41]:
df

,siren,legal_name,id,url_article,corpus,eval_set,eval_number
0,419838529,IPSEN,8,http://www.boursier.com/actions/actualites/new...,"{peptide, ipsen, peptimimesi, le, lorgne, de}",{ipsen},1
1,419838529,IPSEN,2894,http://www.boursier.com/actions/actualites/new...,"{déposer, gastroentérologie, stratégique, le, ...",{ipsen},1
2,419838529,IPSEN,3057,http://www.boursier.com/actions/actualites/new...,"{explique, portefeuille, le, daffair, inscrive...",{ipsen},1
3,419838529,IPSEN,4208,http://www.cerclefinance.com/default.asp?pub=v...,"{boursiere, economie, information, cerclefinan...",{},0
4,419838529,IPSEN,5284,http://www.cerclefinance.com/default.asp?pub=v...,"{boursiere, economie, information, cerclefinan...",{},0
...,...,...,...,...,...,...,...
495,395030844,SANOFI,41863,https://www.lemondedudroit.fr/deals/57157-clif...,"{obligataire, banque, par, clifford, lémission...",{sanofi},1
496,395030844,SANOFI,42528,https://www.lesechos.fr/industrie-services/pha...,"{dun, devise, selon, via, combiné, le, sétabli...","{pharmaceutique, sanofi}",2
497,395030844,SANOFI,43721,https://www.lesechos.fr/industrie-services/pha...,"{respiratoire, dun, paramètre, selon, médiateu...",{sanofi},1
498,395030844,SANOFI,43756,https://www.mypharma-editions.com/sanofi-le-do...,"{recherche, linfo, politique, biologique, glob...","{pharmaceutique, sanofi}",2


In [30]:
#df=df.sample(10)

In [31]:
%%time
df['corpus'] = df['corpus'].map(lambda text: preprocessing(text))
df['corpus'] = df['corpus'].map(lambda text: "".join(text))
df['corpus'] = df['corpus'].map(lambda text: set(text.split()))
df['eval_set'] = ""



CPU times: user 1min, sys: 243 ms, total: 1min
Wall time: 1min 3s


In [32]:
# Get name of all companies in eval dataset 
vocab = set(df.legal_name)


In [33]:
from collections import defaultdict
nearest = defaultdict(str)
# Very important to use lower here, as we lower everything during preprocessing
for x in vocab:
    nearest[x.lower()] = model.get_nearest_neighbors(x.lower(),5)

In [34]:
df['eval_number'] = ""
df['eval_set'] = ""

In [35]:
for i,row in df.iterrows():
    nearest_words = set([y[1:len(y)-2] for x,y in (nearest[row.legal_name.lower()])])
    inters_ = nearest_words.intersection(row.corpus)
    df['eval_set'][i] =  inters_
    df['eval_number'][i] = len(inters_)


<ipython-input-35-d9f942819e8d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['eval_set'][i] =  inters_
<ipython-input-35-d9f942819e8d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['eval_number'][i] = len(inters_)


In [40]:
df.sample(10)

,siren,legal_name,id,url_article,corpus,eval_set,eval_number
362,332822485,HAULOTTE GROUP,73563,https://acteursdeleconomie.latribune.fr/strate...,"{ralentissement, malgré, dégradé, niveau, anné...",{haulotte},1
305,552030967,VALEO,10599,http://www.leparisien.fr/automobile/automobile...,"{intelligent, réaliser, expliqueton, dun, selo...",{valeo},1
216,542107651,ENGIE,27613,https://www.lesechos.fr/industrie-services/ene...,"{réaliser, dun, via, acquisition, ambition, ex...",{engie},1
450,395030844,SANOFI,16270,http://www.boursier.com/actions/actualites/rum...,"{participer, le, de, production, gestion, heur...",{sanofi},1
76,399258755,SPIE OPERATIONS,3187,http://www.boursier.com/actions/actualites/new...,"{commenter, contraction, acquisition, le, insc...",{},0
129,399258755,SPIE OPERATIONS,53632,http://www.boursier.com/actions/actualites/new...,"{lienz, acquisition, dénergie, le, inscrivezvo...",{},0
363,332822485,HAULOTTE GROUP,75943,http://www.boursier.com/https://www.boursier.c...,"{réaliser, ralentissement, depuis, le, inscriv...","{haulotte, groupe}",2
286,542107651,ENGIE,68041,http://www.boursier.com/actions/actualites/new...,"{le, chauffe, y, projet, inscrivezvou, de, ges...",{engie},1
205,542107651,ENGIE,24752,http://www.boursier.com/actions/actualites/new...,"{voie, créer, le, regazéification, projet, nat...",{engie},1
43,572060333,MERSEN,37801,https://www.usinenouvelle.com/article/mersen-v...,"{ayvelle, permet, demploi, depuis, créer, le, ...",{mersen},1


In [37]:
nearest

defaultdict(str,
            {'engie': [(0.8292605876922607, "'engie',"),
              (0.7183860540390015, "'conglomérat',"),
              (0.7017418742179871, "'consortium',"),
              (0.6990586519241333, "'arcelormittal',"),
              (0.6956524848937988, "'saint-gobain',")],
             'spie operations': [(0.9029197692871094, "'operation',"),
              (0.8648870587348938, "'operationnel',"),
              (0.8618251085281372, "'operating',"),
              (0.8363695740699768, "'generation',"),
              (0.8174439668655396, "'preceding',")],
             'banque palatine': [(0.8770163059234619, "'palatine',"),
              (0.8490776419639587, "'palatin',"),
              (0.8435850143432617, "'banque',"),
              (0.8140552043914795, "'edicourtage',"),
              (0.8126886487007141, "'banquier',")],
             'gl events': [(0.804206907749176, "'pricewaterhousecooper',"),
              (0.7970137000083923, "'gentsid',"),
              (0.79346

In [39]:
import numpy as np
np.mean(df['eval_number'])

0.806